vacccination by age is analysised and dash implemented by Bratati Chakrabori

This Python code addresses a situation where a CSV file has a date column containing only "day" values (possibly for test data) and a vaccinated indicator. The code performs two key actions:

Year Extraction: It extracts the year from the date column. Since the column holds just "day" values, conversion to full dates might not be necessary. However, for future compatibility or handling unexpected formats, the code attempts date conversion (with error handling) and extracts the year if successful. Missing values (NaN) are gracefully handled by replacing them with a placeholder value (e.g., -1).
Grouping by Year: It groups the data in the DataFrame by the extracted year. This allows you to analyze trends or patterns within each year based on the other columns in the CSV file.
Explanation:

The code assumes you have pandas (import pandas as pd) installed. It then:

Loads the CSV data into a DataFrame.
Extracts the year from the date column (handling potential errors and missing values).
Groups the DataFrame by the extracted year.
You can then use various pandas methods to analyze or visualize the grouped data.

In [13]:
# Install required libraries (assuming you haven't already)
!pip install -U kaleido dash pandas seaborn

# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
import base64
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Read data from your CSV file (replace 'vaccinations-by-age-group.csv' with the actual path)
df = pd.read_csv('vaccinations-by-age-group.csv')  # Assuming CSV is in the same directory

# Data cleaning and transformation (if needed)
# Assuming dates are in 'DD/MM/YYYY' format, convert to datetime
if 'date' in df.columns:  # Check if 'date' column exists
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
    # Extract year
    df['year'] = df['date'].dt.year

# Define visualization function (using Plotly Express)
def create_visualization(selected_year, selected_country):
    filtered_df = df[df['year'] == selected_year]
    if selected_country:
        filtered_df = filtered_df[df['location'] == selected_country]

    fig = px.bar(
        filtered_df,
        x="age_group",
        y=["people_vaccinated_per_hundred", "people_fully_vaccinated_per_hundred", "people_with_booster_per_hundred"],
        barmode="group",  # Stack bars to show cumulative values
        title=f"Vaccination Rates in {selected_country or 'All Countries'} ({selected_year})"
    )
    fig.update_layout(legend_title_text="Vaccination Status")
    fig.update_traces(marker_line_color='black', marker_line_width=1)  # Improve bar visibility

    # Convert plot to PNG for Dash usage
    img_bytes = BytesIO()
    fig.write_image(img_bytes, format='png')
    img_bytes.seek(0)
    encoded_img = base64.b64encode(img_bytes.getvalue()).decode('utf-8')

    return f'data:image/png;base64,{encoded_img}'

# Initialize Dash app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1("Vaccination Rates by Country and Age Group"),
    html.Div([
        dcc.Dropdown(
            id="year-dropdown",
            options=[{'label': str(y), 'value': y} for y in df['year'].unique()],
            value=df['year'].max()  # Pre-select the latest year
        ),
        dcc.Dropdown(
            id="country-dropdown",
            options=[
                {'label': c, 'value': c} for c in df['location'].unique()] # + [{'label': 'All Countries', 'value': None}],  # Allow filtering by all countries
            #value=None
        )
    ]),
    html.Div(id='graph-container'),
])

# Callback to update the graph
@app.callback(
    Output(component_id='graph-container', component_property='children'),
    [Input(component_id='year-dropdown', component_property='value'),
     Input(component_id='country-dropdown', component_property='value')]
)
def update_graph(selected_year, selected_country):
    graph_image = create_visualization(selected_year, selected_country)
    return html.Img(src=graph_image)

if __name__ == '__main__':
    app.run_server(debug=True)
